<a href="https://colab.research.google.com/github/pernilletemmerud/DCF-PT/blob/main/CodingTask1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Science Fundamentals for DCBP, S. Haug, University of Bern 

# Coding Task 1

- Deadline (upload to Ilias): 2022-03-20 24:00
- Estimated effort: 2-5 hours
- Data on Ilias (4ms_...) with metadata file CCD-Metadata


## Tasks (maximum 12.5 points)

- Write a method (function) which removes the header information in the datafile and saves that information into a separate textfile. [0.5 points] 
- Write a function which reduces the data resolution by averaging over 1, 2, 3 and 4 nm wavelengths. Save these 4 new reduced datasets in new files. Do this (1) by writing an explicit loop and (2) by using pandas methods. Measure the running times of the two approaches. [6 points]
- Write a function which takes two wavelengths as input and plots the difference of the data at these lengths over time. [2 points] 
- Let the x-axis be in seconds, label the axes with names and units, label the plot with color and legend, make a title for the plot. Save the plot to a file. [2 points]
- Plot the full and the reduced datasets in the same figure. Is there any visual difference? [2 points]

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# mounting the file to the notebook
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path = '/content/drive/MyDrive/DataScienceFundamentals/data_codingtask_1.txt'
df = pd.read_csv(path, delimiter = '\t', header=13)
df.head()

Mounted at /content/drive


,Unnamed: 0,Unnamed: 1,316.6137,316.7492,316.8847,317.0203,317.1558,317.2913,317.4268,317.5623,...,778.9703,779.089,779.2076,779.3263,779.4449,779.5636,779.6822,779.8008,779.9194,780.0381
0,2018-10-25 10:32:04.789000,1.540456e+12,0.329672,0.329672,0.329672,0.090294,-0.716926,-1.829304,-0.686330,-0.428776,...,-0.038483,-0.039623,-0.036920,-0.037309,-0.037299,-0.037476,-0.036856,-0.036853,-0.039100,-0.035545
1,2018-10-25 10:32:04.878000,1.540456e+12,-0.044412,-0.044412,-0.044412,0.157632,-0.672423,-2.183981,-0.492421,-0.000000,...,-0.039428,-0.038486,-0.037309,-0.036357,-0.037289,-0.038728,-0.037214,-0.036902,-0.040210,-0.038165
2,2018-10-25 10:32:04.962000,1.540456e+12,0.338219,0.338219,0.338219,0.330993,-0.879836,-0.000000,-0.559845,-0.000000,...,-0.037725,-0.037534,-0.036560,-0.036152,-0.036554,-0.036875,-0.037351,-0.036746,-0.038686,-0.036903
3,2018-10-25 10:32:05.046000,1.540456e+12,-0.049682,-0.049682,-0.049682,0.519086,-0.681150,-0.000000,-0.417230,-0.000000,...,-0.037613,-0.037168,-0.036369,-0.034549,-0.036252,-0.037948,-0.036713,-0.035706,-0.038203,-0.036822
4,2018-10-25 10:32:05.129000,1.540456e+12,0.335086,0.335086,0.335086,-0.041393,-0.000000,-1.802774,-0.510417,-0.000000,...,-0.040919,-0.038288,-0.035840,-0.035638,-0.039504,-0.040093,-0.039953,-0.037656,-0.038082,-0.035900


In [8]:
def save_header(p, n):   #p = path, n = new file name
  header = ' '     
  with open (path, 'r') as f:  #opening the file in reading mode
    lines = f.readlines()  
    for i in range (0,13):
      header += lines[i]  
  f = open(n,'w+')  #creating a new file 
  f.write(str(header))
  f.close()

def save_data(p, n):     #p = path, n = new file name
  df = pd.read_csv(p, delimiter='\t', header=13)  #opening the datafile
  f = open(n, 'w+')     #creating a new file
  f.write(str(df))
  f.close()      
    
save_header(path, 'header.txt')
save_data(path, 'data.csv')

In [9]:
test = open('header.txt', 'r')  #checking if the file was created correctly
print(test.read())  

 Data from 4ms_10 av_15 s_Absorbance_10-32-04-868.txt Node

Date: Thu Oct 25 10:32:04 CEST 2018
User: Christoph
Spectrometer: BALLMOOS
Trigger mode: 4
Integration Time (sec): 4.000000E-3
Scans to average: 10
Electric dark correction enabled: true
Nonlinearity correction enabled: false
Boxcar width: 0
XAxis mode: Wavelengths
Number of Pixels in Spectrum: 3648



In [ ]:
Write a function which reduces the data resolution by averaging over 1, 2, 3 and 4 nm wavelengths. Save these 4 new reduced datasets in new files. 
Do this (1) by writing an explicit loop and (2) by using pandas methods. Measure the running times of the two approaches. [6 points]

In [ ]:
def loop_average(): 
  







In [ ]:
def pandas_average(): 






  

In [15]:
# the wavelengths are the header of the dataframe
'''
def reduce_resolution(r, df):    #r = the number we want to average over, df = the original dataframe
  wavelengths = df.columns.values
  wavelengths = wavelengths[2:]

  for w in wavelengths: 
    old_value = float(w)
    new_value = old_value/r
    new_df = df.rename(columns={old_value:new_value}) # funker ikke å erstatte med ny verdi

  return new_df

new_df = reduce_resolution(2, df)

new_df.head()
'''

'\ndef reduce_resolution(r, df):    #r = the number we want to average over, df = the original dataframe\n  wavelengths = df.columns.values\n  wavelengths = wavelengths[2:]\n\n  for w in wavelengths: \n    old_value = float(w)\n    new_value = old_value/r\n    new_df = df.rename(columns={old_value:new_value}) # funker ikke å erstatte med ny verdi\n\n  return new_df\n\nnew_df = reduce_resolution(2, df)\n\nnew_df.head()\n'